In [4]:
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import numpy as np
import os
import matplotlib.pyplot as plt
import os
import time
import glob
from keras.models import Sequential
import fnmatch
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

DATA = os.getenv('DEXNET_DATA')
print(DATA)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1501853929446138523
]
Num GPUs Available:  0
/home/firefly/dexnet/dexnet_2/dexnet_2_tensor/tensors/


In [15]:
import tensorflow as tf
import math
import random

# example of defining the discriminator model
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Lambda
from tensorflow.nn import local_response_normalization
from keras.utils.vis_utils import plot_model

from keras.layers.merge import concatenate
from keras.models import Model
# from tensorflow.keras import layers
from keras.layers import Dense, Input


#TODO: Initialize weights in layers according to the paper https://arxiv.org/pdf/1703.09312.pdf
def getGraspQualityVariable():
    input = Input(shape=(32, 32, 1), name="img")
    x1 = Conv2D(filters=64, kernel_size=7, activation='relu')(input)
    x2 = Conv2D(filters=64, kernel_size=5, activation='relu')(x1)
    x3 = Lambda(local_response_normalization)(x2)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=2)(x3)
    x5 = Conv2D(filters=64, kernel_size=3, activation='relu')(x4)
    # x6 = Dropout(0.3)(x5)
    x6 = Conv2D(filters=64, kernel_size=3, activation='relu')(x5)
    x7 = Lambda(local_response_normalization)(x6)
    x8 = Flatten()(x7)
    x9 = Dense(1024, activation='relu')(x8)
    
    # plot_model(model, to_file='GraspQualityModel_plot.png', show_shapes=True, show_layer_names=True)
    return x9, input
    
getGraspQualityVariable()


def getPointcloudModel():
    input = Input(shape=(1), name="z")
    x = Dense(16, input_dim=1, activation='relu')(input)
    return x, input


from keras.layers.merge import concatenate
def getDexnet2Model():
    grasp_model, input_1 = getGraspQualityVariable()
    pc_model, input_2 = getPointcloudModel()

    out = Dense(1024, activation='relu')

    # x = layers.concatenate([grasp_model, pc_model])
    merge = concatenate([grasp_model, pc_model])
    out_1 = Dense(1024, activation='relu')(merge)
    out_2 = Dense(2, activation='softmax')(out_1)
    model = Model(inputs=[input_1, input_2], outputs=out_2)
    plot_model(model, to_file='Dexnet2_plot.png', show_shapes=True, show_layer_names=True)
    return model

In [20]:

aligned_imgs_files = []
for filename in os.listdir(DATA):
    if fnmatch.fnmatch(filename, 'depth_ims_tf_table_*.npz'):
        aligned_imgs_files.append(filename)  # collected all the depth_ims_tf_table files in one list
aligned_imgs_files = np.sort(aligned_imgs_files)
print(aligned_imgs_files[10:15])

gripper_depths_files = []
for filename in os.listdir(DATA):
    if fnmatch.fnmatch(filename, 'hand_poses_*.npz'):
        gripper_depths_files.append(filename)  # collected all hand_poses the files in one list
gripper_depths_files = np.sort(gripper_depths_files)
print(gripper_depths_files[10:15])

grasp_metrics_files = []
for filename in os.listdir(DATA):
    if fnmatch.fnmatch(filename, 'robust_ferrari_canny_*.npz'):
        grasp_metrics_files.append(filename)  # collected all the grasp_metrics files in one list
grasp_metrics_files = np.sort(grasp_metrics_files)
print(grasp_metrics_files[10:15])


#------------------------------------------------------------------------------------------------------------------#    
 
for i in range (10):

    X1 = [np.load(DATA + filename) for fname in aligned_imgs_files[100*i:100*(i+1)]]
    
    X2 = [np.load(DATA + filename) for fname in gripper_depths_files[100*i:100*(i+1)]]
    
    Y  = [np.load(DATA + filename) for fname in grasp_metrics_files[100*i:110*(i+1)]]
#######################      
    merged_X1 = []
    for data in X1:
        merged_X1.append(X1['arr_0.npy'])

    del data

    concatenated_merged_X1 = merged_X1[0]
    for  c in range(1,100): 
            
        concatenated_merged_X1 = np.concatenate([concatenated_merged_X1,merged_X1[c]])
        print(c)
        print(np.shape(concatenated_merged_X1))
    
    del merged_X1

########################
    merged_X2 = []
    for data in X1:
        merged_X2.append(X1['arr_0.npy'])

    del data

    concatenated_merged_X2 = merged_X2[0]
    for  c in range(1,100): 
            
        concatenated_merged_X2 = np.concatenate([concatenated_merged_X2,merged_X2[c]])
        print(c)
        print(np.shape(concatenated_merged_X2))
    
    del merged_X2
#######################
    merged_Y = []
    for data in Y:
        merged_Y.append(Y['arr_0.npy'])

    del data

    concatenated_merged_Y = merged_Y[0]
    for  c in range(1,100): 
            
        concatenated_merged_Y = np.concatenate([concatenated_merged_Y,merged_Y[c]])
        print(c)
        print(np.shape(concatenated_merged_Y))
    
    del merged_Y


    aligned_imgs  = concatenated_merged_X1
    gripper_depths = concatenated_merged_X2
    grasp_metrics = concatenated_merged_Y


    #******************One hot encoding***********************

    # print(grasp_metrics[2])
    def one_hot_encoding(grasp_metrics):
        for i in range(1000):
            if(grasp_metrics[i] > 0.002):  #threshold value is 0.002 readme
                grasp_metrics[i] = 1
            else:
                grasp_metrics[i] = 0
        return grasp_metrics

    grasp_metrics = one_hot_encoding(grasp_metrics)

    #------------------------------------------------------------------------------------------#

    first_model = getDexnet2Model()

    first_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


    from keras.callbacks import ModelCheckpoint
    start_time = time.time()
    checkpointer = ModelCheckpoint(filepath='/home/firefly/dexnet/dexnet2_git/Weights/second_model.weights.best.hdf5', verbose = 1, save_best_only=True)
    x_train = [aligned_imgs, gripper_depths]                                   #Check model.summary() in previous section and check model architecture in paper
    y_train = grasp_metrics                                                    #valued between [0, 1] grasp robustness for the given grasp
    first_model.fit(x_train,
            y_train,
            batch_size=8,                                                   #worth ecperimenting
            epochs=10,                                      
            validation_split=0.25,                                            #Decide on a number
            callbacks=[checkpointer])        
    #CHange the filepath of the checkpointer varianble to store different version of weights
    elapsed_time = (time.time() - start_time)
    print(f"--- {elapsed_time} seconds ---")
    print(f"--- {elapsed_time/60} minutes ---")
    print(f"--- {elapsed_time/3600} hours ---")


    



['depth_ims_tf_table_00010.npz' 'depth_ims_tf_table_00011.npz'
 'depth_ims_tf_table_00012.npz' 'depth_ims_tf_table_00013.npz'
 'depth_ims_tf_table_00014.npz']
['hand_poses_00010.npz' 'hand_poses_00011.npz' 'hand_poses_00012.npz'
 'hand_poses_00013.npz' 'hand_poses_00014.npz']
['robust_ferrari_canny_00010.npz' 'robust_ferrari_canny_00011.npz'
 'robust_ferrari_canny_00012.npz' 'robust_ferrari_canny_00013.npz'
 'robust_ferrari_canny_00014.npz']


TypeError: list indices must be integers or slices, not str

In [15]:
grasp_metrics[290:300]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])